In [1]:
import sys
sys.path.insert(1, '../')

from river import stream,tree,metrics
import utils
from encoding import prefix_bin
import csv
import copy
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from tqdm import tqdm
import pickle as pkl
import matplotlib.pyplot as plt
import os,json
import datetime
from collections import deque
from sklearn.tree import DecisionTreeClassifier

import pandas as pd

In [2]:
def performance_measurement(metrics, type='Accuracy'):
    if type == 'Accuracy':
        return_value = metrics.Accuracy()
    elif type =='F1':
        return_value = metrics.F1()
    elif type =='ROCAUC':
        return_value = metrics.ROCAUC()
    elif type == 'WeightedF1':
        return_value = metrics.WeightedF1()
        
    return return_value

In [3]:
def display_progress(row_counting, total_length, interval=2000):
    if rowcounter%interval == 0:
        print(round(rowcounter*100/totallength,2) ,'%', 'Case finished: %s'%(casecount), 'Running cases: %s'%(len(streaming_db)))

In [4]:
def update_test_cases(new_case, test_cases):
    test_cases.append(new_case)
    if len(test_cases) > test_size:
        test_cases.popleft()

In [16]:
# Experiment settings
'./DATA/logs/synthetic_log_bc1.csv'
dataset_label = 'bpic17'
monitoring = 'updating'


# Invoke parameters for dataset
window_size = 100
test_size = 30
with open('../dataset_parameters.json','r') as json_file:
    parameters = json.load(json_file)[dataset_label]
    print(parameters)
    key_pair = parameters['key_pair']
    catatars = parameters['categorical_attrs']
    maximum_prefixs = parameters['maximum_prefix']

maximum_prefixs =2
for maximum_prefix in range(1, maximum_prefixs+1):

    training_windows = []

    print(maximum_prefix)
    training_window_dict = dict()
    test_window_dict = dict()
    
    dataset_loc = '../DATA/logs/'+ dataset_label +'.csv'
    try:
        os.makedirs('../result/%s'%(dataset_label))
    except:
        pass

    # Set streaming classifier
    classifier = 'efdt'
    if classifier == 'htc':
        streaming_classifier = tree.HoeffdingTreeClassifier(grace_period=10, split_criterion='info_gain')
    elif classifier =='hatc':
        streaming_classifier = tree.HoeffdingAdaptiveTreeClassifier(grace_period=10, split_criterion='info_gain' )
    elif classifier == 'efdt':
        streaming_classifier = tree.ExtremelyFastDecisionTreeClassifier(grace_period=10, split_criterion='info_gain')
        
    dataset = stream.iter_csv(
            dataset_loc
            )

    totallength = len(list(dataset))

    dataset = stream.iter_csv(
            dataset_loc,
            # drop=['Complete Timestamp'],
            target='outcome'
            )
    enctype = 'Index-base'
    
    streaming_db ={}
    training_models ={}
    test_cases = deque()
    feature_matrix ={}

    save_model = {}
    casecount = 0
    rowcounter = 0
    finished_db ={}
    running_case = 0
    window_acc_dict = {}
    prediction_result = dict()
    for i in range(1, maximum_prefix+1): prediction_result[i] = {}
    finished_caseid = set()
    usedingrace = set()
    
    
    '''
    Before test and training streaming event predictive monitoring, grace period is preceded to initialize model by prefix length
    and obtain feature matrix to transform future events
    '''
    for x,y in dataset:
#         display_progress(rowcounter, totallength)
        rowcounter +=1
        # Event stream change dictionary keys
        x = utils.dictkey_chg(x, key_pair)

        if dataset_label !='bpic15':
            x['ts'] = x['ts'][:-4]

        x['outcome'] =y 
        # Initialize case by prefix length        
        caseid = x['caseid']
        outcome = x['outcome']
    #     progress = x['progress']

        x.pop('caseid')
        x.pop('outcome')

    #     x.pop('progress')

        case_bin = prefix_bin(caseid, x)
        case_bin.set_enctype(enctype)

        if caseid not in list(streaming_db.keys()):
            case_bin.set_prefix_length(1)    
            streaming_db[caseid] = []
        elif caseid in finished_caseid:
            pass
        else:
            case_bin.set_prefix_length(len(streaming_db[caseid])+1)
            case_bin.set_prev_enc(streaming_db[caseid][-1])

        # Encode event and cases and add to DB
        case_bin.update_truelabel(outcome)   
        case_bin.update_encoded(catattrs=catatars,enctype=enctype)
        ts = case_bin.event['ts']
        streaming_db[caseid].append(case_bin)
        # Detect label appeared case 
        if outcome != '' and caseid not in finished_caseid:
            usedingrace.add(caseid)
            for i in streaming_db[caseid]:
                i.update_truelabel(outcome)
            finished_caseid.add(caseid)
            # Adding newly finished case to training set.    
            casecount +=1
            # Grace period to collect feature matrix
            if casecount < 200:
                case_length = len(streaming_db[caseid])
                if case_length >= maximum_prefix:
                    if 'prefix_%s'%(maximum_prefix) not in list(feature_matrix.keys()):
                        feature_matrix['prefix_%s'%(maximum_prefix)]=set()
                        training_models['prefix_%s'%(maximum_prefix)] = [streaming_classifier,
                                                                   0]
                    feature_list = list(streaming_db[caseid][maximum_prefix-1].encoded.keys())
                    for x in feature_list: feature_matrix['prefix_%s'%(maximum_prefix)].add(x) 

            else:
                break
    casecount_=0
    for caseid in list(usedingrace):

        case_length = len(streaming_db[caseid])
        if case_length >= maximum_prefix:
            x = streaming_db[caseid][maximum_prefix-1]
            if x.prefix_length != 0:            
                x.encoded = utils.readjustment_training(x.encoded, feature_matrix['prefix_%s'%(maximum_prefix)])
                outcome = x.true_label
                training_models['prefix_%s'%(x.prefix_length)][0].learn_one(x.encoded,outcome)
                training_models['prefix_%s'%(x.prefix_length)][1] +=1
                training_windows.append(streaming_db[caseid][maximum_prefix-1])
            update_test_cases(streaming_db[caseid], test_cases)

        prediction_result[maximum_prefix][casecount_] = {}
        y_truelist = []
        y_predlist = []
        for case in test_cases:
            if len(case) >= maximum_prefix:
                x = case[maximum_prefix-1]
                if x.prefix_length != 0:            
                    model = training_models['prefix_%s'%(x.prefix_length)][0]
                    y_pred = training_models['prefix_%s'%(x.prefix_length)][0].predict_proba_one(x.encoded)
                    y_truelist.append(x.true_label)
                    y_predlist.append(y_pred)
        prediction_result[maximum_prefix][casecount_]['y_true'] = y_truelist
        prediction_result[maximum_prefix][casecount_]['y_pred'] = y_predlist
        casecount_ +=1
    
    '''
    Streaming event label prediction start.
    - Test and training steps are executed when case finished/ event arrived with label
    '''
    for i in streaming_db.keys(): usedingrace.add(i)
    streaming_db ={}
    cdhappend ={}
    for i in range(1, maximum_prefix+1): cdhappend[i] = 0
    for x,y in dataset:
#         display_progress(rowcounter, totallength)

        rowcounter +=1
        # Event stream change dictionary keys
        x = utils.dictkey_chg(x, key_pair)

        if dataset_label !='bpic15':
            x['ts'] = x['ts'][:-4]

        # Check label possible
        # x = utils.set_label(x)
        x['outcome'] =y 
        # Initialize case by prefix length
        caseid = x['caseid']
        outcome = x['outcome']
        x.pop('caseid')
        x.pop('outcome')

        if caseid not in usedingrace:
            case_bin = prefix_bin(caseid, x)
            case_bin.set_enctype(enctype)

            if caseid not in list(streaming_db.keys()):
                case_bin.set_prefix_length(1)    
                streaming_db[caseid] = []
                running_case +=1
            elif caseid in finished_caseid:
                pass
            else:
                case_bin.set_prefix_length(len(streaming_db[caseid])+1)
                case_bin.set_prev_enc(streaming_db[caseid][-1])

            # Encode event and cases and add to DB
            case_bin.update_truelabel(outcome)   
            case_bin.update_encoded(catattrs=catatars,enctype=enctype)
            ts = case_bin.event['ts']

            if case_bin.prefix_length == maximum_prefix:
                case_bin.encoded = utils.readjustment_training(case_bin.encoded, feature_matrix['prefix_%s'%(case_bin.prefix_length)])
            streaming_db[caseid].append(case_bin)

            # Detect label appeared case 
            if outcome != '' and caseid not in finished_caseid:
                finished_caseid.add(caseid)

                # Adding newly finished case to training set.
                casecount +=1    

                # Modify encoded attributes of cases with feature matrix
                case_length = len(streaming_db[caseid])
                if case_length >= maximum_prefix:

                    update_test_cases(streaming_db[caseid], test_cases)
                    streaming_db[caseid][maximum_prefix-1].update_truelabel(outcome)
                    x = streaming_db[caseid][maximum_prefix-1].encoded
                    prefix_length =streaming_db[caseid][maximum_prefix-1].prefix_length
                    training_models['prefix_%s'%(prefix_length)][0].learn_one(x, outcome)
                    training_models['prefix_%s'%(prefix_length)][1] +=1

                    y_pred = training_models['prefix_%s'%(prefix_length)][0].predict_one(x)
                    model_update_count = training_models['prefix_%s'%(prefix_length)][1]
                    
                    save_model['%s_%s'%(prefix_length, model_update_count)] = training_models['prefix_%s'%(streaming_db[caseid][maximum_prefix-1].prefix_length)][0]

                    training_windows.append(streaming_db[caseid][maximum_prefix-1])
                    training_window_dict[casecount] = copy.deepcopy(training_windows)
                    test_window_dict[casecount] = copy.deepcopy([c[0] for c in test_cases if len(c) >= maximum_prefix ])

                y_truelist = []
                y_predlist = []

                for case in test_cases:
                    if len(case) >= maximum_prefix:
                        x = case[maximum_prefix-1]
                        if x.prefix_length != 0:
                            length = x.prefix_length
                            y_pred = training_models['prefix_%s'%(x.prefix_length)][0].predict_proba_one(x.encoded)
                            y_truelist.append(x.true_label)
                            y_predlist.append(y_pred)

                prediction_result[maximum_prefix][casecount] = {}
                prediction_result[maximum_prefix][casecount]['y_true'] = y_truelist
                prediction_result[maximum_prefix][casecount]['y_pred'] = y_predlist
                if 'b1' not in caseid and cdhappend[maximum_prefix] == 0:
                    cdhappend[maximum_prefix] = model_update_count

    label_dist_dict = {}
    label_dist_dict[0] = {'False': 0,'True':0}
    for t in range(1, casecount+1):
        if t not in training_window_dict.keys():
            label_dist_dict[t] = label_dist_dict[t-1]
        else:            
            window = training_window_dict[t]
            label_dist_dict[t] = {'False': 0,'True':0}
            for i in range(len(window)):
                if window[i].true_label == 'False':
                    label_dist_dict[t]['False'] +=1
                else:
                    label_dist_dict[t]['True'] +=1
                    
    dist_df = pd.DataFrame.from_dict(label_dist_dict).T
    dist_df = dist_df.reset_index(drop=False)
    dist_df = dist_df.rename(columns={'index':'Case count'})
    print(dist_df)
    dist_df.to_csv('../result/bpic17/%s_prefix_%s_%s_training_window_label.csv'%(classifier, maximum_prefix, monitoring), index=False)

{'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Resource': 'resource', 'Start Timestamp': 'ts'}, 'categorical_attrs': ['activity', 'resource'], 'maximum_prefix': 15}
1


NameError: name 'pd' is not defined

In [8]:
len(training_window_dict[201])

201

In [13]:
training_window_dict[201][3].caseid

'Application_617110337'